# PSP Network
Pyramid Scene Parsing Network

- [paper: Pyramid Scene Parsing Network](https://arxiv.org/abs/1612.01105)
- [github: PyTorch Semantic Segmentation](https://github.com/hszhao/semseg)

## Contributions
- Propose a pyramid scene parsing network to embed difficult scenery context features in an FCN based pixel prediction framework.
- Develop an effective optimization strategy for deep ResNet based on deeply supervised loss.
    - Use auxiliary loss and master branch's softmax loss

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from pycocotools.coco import COCO
import torchvision.models as models

In [2]:
import os
import random
import time
import cv2
import json
import logging
from datetime import datetime

import numpy as np
import pandas as pd

import torch.backends.cudnn as cudnn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.multiprocessing as mp
import torch.distributed as dist
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [3]:
import wandb
run = wandb.init(project='p3-img-seg', entity='boostcamp-simple')

wandb: Currently logged in as: jaegyeong (use `wandb login --relogin` to force relogin)


In [4]:
print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.4.0
GPU 사용 가능 여부: True
Tesla P40
1


## 하이퍼파라미터 세팅 및 seed 고정

In [5]:
config = wandb.config

config.update({
    "batch_size": 16,
    "num_epochs": 10,
    "dropout": 0.2,
    "architecture": "PSP with ResNet",
    "random_seed": 21,

    "weight_decay": 0.0001,
    "momentum": 0.9,
    "zoom_factor": 8,
    "aux_weight": 0.4,
    "ignore_label": 255,
    "base_lr": 0.01,
    "power": 0.9,
})

In [6]:
# seed 고정
random_seed = config.random_seed

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
# torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## 데이터 전처리 함수 정의 (Dataset)

In [7]:
dataset_path = '/opt/ml/input/data'
anns_file_path = dataset_path + '/' + 'train.json'

# Read annotations
with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())

categories = dataset['categories']
anns = dataset['annotations']
imgs = dataset['images']
nr_cats = len(categories)
nr_annotations = len(anns)
nr_images = len(imgs)

# Load categories and super categories
cat_names = []
super_cat_names = []
super_cat_ids = {}
super_cat_last_name = ''
nr_super_cats = 0
for cat_it in categories:
    cat_names.append(cat_it['name'])
    super_cat_name = cat_it['supercategory']
    # Adding new supercat
    if super_cat_name != super_cat_last_name:
        super_cat_names.append(super_cat_name)
        super_cat_ids[super_cat_name] = nr_super_cats
        super_cat_last_name = super_cat_name
        nr_super_cats += 1

print('Number of super categories:', nr_super_cats)
print('Number of categories:', nr_cats)
print('Number of annotations:', nr_annotations)
print('Number of images:', nr_images)

Number of super categories: 11
Number of categories: 11
Number of annotations: 21116
Number of images: 2617


In [8]:
# Count annotations
cat_histogram = np.zeros(nr_cats,dtype=int)
for ann in anns:
    cat_histogram[ann['category_id']] += 1

# Convert to DataFrame
df = pd.DataFrame({'Categories': cat_names, 'Number of annotations': cat_histogram})
df = df.sort_values('Number of annotations', 0, False)

In [9]:
# category labeling 
sorted_temp_df = df.sort_index()

# background = 0 에 해당되는 label 추가 후 기존들을 모두 label + 1 로 설정
sorted_df = pd.DataFrame(["Backgroud"], columns = ["Categories"])
sorted_df = sorted_df.append(sorted_temp_df, ignore_index=True)

In [10]:
category_names = list(sorted_df.Categories)

def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images /= 255.0
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id + 1" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # Unknown = 1, General trash = 2, ... , Cigarette = 11
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = category_names.index(className)
                masks = np.maximum(self.coco.annToMask(anns[i])*pixel_value, masks)
            masks = masks.astype(np.float32)

            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            
            return images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

## Dataset 정의 및 DataLoader 할당

In [11]:
# train.json / validation.json / test.json 디렉토리 설정
train_path = dataset_path + '/train.json'
val_path = dataset_path + '/val.json'
test_path = dataset_path + '/test.json'

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

train_transform = A.Compose([
                            A.Resize(473, 473),
                            ToTensorV2(),
                            ])

val_transform = A.Compose([
                          A.Resize(473, 473),
                          ToTensorV2(),
                          ])

test_transform = A.Compose([
                           A.Resize(473, 473),
                           ToTensorV2()
                           ])

# create own Dataset 1 (skip)
# validation set을 직접 나누고 싶은 경우
# random_split 사용하여 data set을 8:2 로 분할
# train_size = int(0.8*len(dataset))
# val_size = int(len(dataset)-train_size)
# dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=transform)
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# create own Dataset 2
# train dataset
train_dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=train_transform)

# validation dataset
val_dataset = CustomDataLoader(data_dir=val_path, mode='val', transform=val_transform)

# test dataset
test_dataset = CustomDataLoader(data_dir=test_path, mode='test', transform=test_transform)


# DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=config.batch_size,
                                           shuffle=True,
                                           num_workers=0,
                                           collate_fn=collate_fn)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                         batch_size=config.batch_size,
                                         shuffle=False,
                                         num_workers=0,
                                         collate_fn=collate_fn)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=config.batch_size,
                                          num_workers=0,
                                          collate_fn=collate_fn)

loading annotations into memory...
Done (t=3.73s)
creating index...
index created!
loading annotations into memory...
Done (t=1.44s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


## PSPNet (ResNet50 imageNet weight)

In [12]:
class PPM(nn.Module):
    """
    Pyramid Pooling Module
    """
    def __init__(self, in_dim, reduction_dim, bins):
        super(PPM, self).__init__()
        self.features = []
        for bin in bins:
            self.features.append(nn.Sequential(
                nn.AdaptiveAvgPool2d(bin),
                nn.Conv2d(in_dim, reduction_dim, kernel_size=1, bias=False),
                nn.BatchNorm2d(reduction_dim),
                nn.ReLU(inplace=True)
            ))
        self.features = nn.ModuleList(self.features)
        
    def forward(self, x):
        x_size = x.size()
        out = [x]
        for f in self.features:
            out.append(F.interpolate(f(x), x_size[2:], mode='bilinear', align_corners=True))
        return torch.cat(out, 1)

In [13]:
class PSPNet(nn.Module):
    def __init__(self, layers=50, bins=(1, 2, 3, 6), dropout=0.1, classes=12, zoom_factor=8, use_ppm=True,
                criterion=nn.CrossEntropyLoss(ignore_index=225), pretrained=True):
        """
        Args
            layer       : depth of resnet
            bins        : pooling kernel size
            dropout     : ratio of dropout
            classes     : number of classes
            zoom_factor : ?
            use_ppm     : Whether or not to use pyramid pooling module
            criterion   : loss function
            pretrained  : Whether or not to use pretrained model
        """
        super(PSPNet, self).__init__()
        assert layers in [50, 101, 152]
        assert 2048 % len(bins) == 0
        assert classes > 1
        assert zoom_factor in [1, 2, 4, 8]
        self.zoom_factor = zoom_factor
        self.use_ppm = use_ppm
        self.criterion = criterion
        
        if layers == 50:
            resnet = models.resnet50(pretrained=pretrained)
        elif layers == 101:
            resnet = models.resnet101(pretrained=pretrained)
        else:
            resnet = models.resnet152(pretrained=pretrained)
            
        self.layer0 = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool)
        self.layer1, self.layer2, self.layer3, self.layer4 = resnet.layer1, resnet.layer2, resnet.layer3, resnet.layer4
        
        # layer3과 layer4의 모든 Convolution에 dilation
        for n, m in self.layer3.named_modules():
            if 'conv2' in n:
                m.dilation, m.padding, m.stride = (2, 2), (2, 2), (1, 1)
            elif 'downsample.0' in n:
                m.stride = (1, 1)
                
        for n, m in self.layer4.named_modules():
            if 'conv2' in n:
                m.dilation, m.padding, m.stride = (4, 4), (4, 4), (1, 1)
            elif 'downsample.0' in n:
                m.stride = (1, 1)
                
        fea_dim = 2048
        if use_ppm:
            self.ppm = PPM(fea_dim, int(fea_dim/len(bins)), bins)
            fea_dim *= 2
            
        self.cls = nn.Sequential(
            nn.Conv2d(fea_dim, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=dropout),
            nn.Conv2d(512, classes, kernel_size=1)
        )
        if self.training:  # Boolean represents whether this module is in training or evaluation mode.
            self.aux = nn.Sequential(
                nn.Conv2d(1024, 256, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm2d(256),
                nn.ReLU(inplace=True),
                nn.Dropout2d(p=dropout),
                nn.Conv2d(256, classes, kernel_size=1)
            )
            
    def forward(self, x, y=None):
        x_size = x.size()
        assert (x_size[2]-1) % 8 == 0 and (x_size[3]-1) % 8 == 0
        h = int((x_size[2] - 1) / 8 * self.zoom_factor + 1)
        w = int((x_size[3] - 1) / 8 * self.zoom_factor + 1)

        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x_tmp = self.layer3(x)
        x = self.layer4(x_tmp)
        if self.use_ppm:
            x = self.ppm(x)
        x = self.cls(x)
        if self.zoom_factor != 1:
            x = F.interpolate(x, size=(h, w), mode='bilinear', align_corners=True)

        if self.training:
            aux = self.aux(x_tmp)
            if self.zoom_factor != 1:
                aux = F.interpolate(aux, size=(h, w), mode='bilinear', align_corners=True)
            main_loss = self.criterion(x, y)
            aux_loss = self.criterion(aux, y)
            return x.max(1)[1], main_loss, aux_loss
        else:
            return x

## train, validation, test 함수 정의

In [14]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [15]:
def intersectionAndUnionGPU(output, target, K, ignore_index=255):
    # 'K' classes, output and target sizes are N or N * L or N * H * W, each value in range 0 to K - 1.
    assert (output.dim() in [1, 2, 3])
    assert output.shape == target.shape
    output = output.view(-1)
    target = target.view(-1)
    output[target == ignore_index] = ignore_index
    intersection = output[output == target]
    area_intersection = torch.histc(intersection.float(), bins=K, min=0, max=K-1)
    area_output = torch.histc(output.float(), bins=K, min=0, max=K-1)
    area_target = torch.histc(target.float(), bins=K, min=0, max=K-1)
    area_union = area_output + area_target - area_intersection
    return area_intersection, area_union, area_target

In [16]:
def poly_learning_rate(base_lr, curr_iter, max_iter, power=0.9):
    """poly learning rate policy"""
    lr = base_lr * (1 - float(curr_iter) / max_iter) ** power
    return lr

In [17]:
def train(train_loader, val_loader, model, criterion, optimizer, epoch, saved_dir):
    main_loss_meter = AverageMeter()
    aux_loss_meter = AverageMeter()
    loss_meter = AverageMeter()
    intersection_meter = AverageMeter()
    union_meter = AverageMeter()
    target_meter = AverageMeter()
    
    classes = 12
    print_freq = 100
    index_split = 0
    val_every = 1
    best_mIoU = 0

    max_iter = config.num_epochs * len(train_loader)
    for i, (input, target, _) in enumerate(train_loader):
        model.train()
        if config.zoom_factor != 8:
            h = int((target.size()[1] - 1) / 8*config.zoom_factor + 1)
            w = int((target.size()[2] - 1) / 8*config.zoom_factor + 1)
            # 'nearest' mode doesn't support align_corners mode and 'bilinear' mode is fine for downsampling
            target = F.interpolate(target.unsqueeze(1).float(), size=(h, w), mode='bilinear', align_corners=True).squeeze(1).long()
        
        input = torch.stack(input)       # (batch, channel, height, width)
        target = torch.stack(target).long()
        
        output, main_loss, aux_loss = model(input, target)
        
        main_loss, aux_loss = torch.mean(main_loss), torch.mean(aux_loss)
        loss = main_loss + config.aux_weight * aux_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        n = input.size(0)
        intersection, union, target = intersectionAndUnionGPU(output, target, classes)
        
        intersection, union, target = intersection.cpu().numpy(), union.cpu().numpy(), target.cpu().numpy()
        intersection_meter.update(intersection), union_meter.update(union), target_meter.update(target)
        
        accuracy = sum(intersection_meter.val) / (sum(target_meter.val) + 1e-10)
        main_loss_meter.update(main_loss.item(), n)
        aux_loss_meter.update(aux_loss.item(), n)
        loss_meter.update(loss.item(), n)
        
        current_iter = epoch * len(train_loader) + i + 1
        current_lr = poly_learning_rate(config.base_lr, current_iter, max_iter, power=config.power)
        for index in range(0, index_split):
            optimizer.param_groups[index]['lr'] = current_lr
        for index in range(index_split, len(optimizer.param_groups)):
            optimizer.param_groups[index]['lr'] = current_lr * 10
            
        
        if (i+1) % print_freq == 1:
            print(('Epoch: [{}/{}][{}/{}] '
                        'MainLoss {main_loss_meter.val:.4f} '
                        'AuxLoss {aux_loss_meter.val:.4f} '
                        'Loss {loss_meter.val:.4f} '
                        'Accuracy {accuracy:.4f}.'.format(epoch+1, config.num_epochs, i+1, len(train_loader),
                                                          main_loss_meter=main_loss_meter,
                                                          aux_loss_meter=aux_loss_meter,
                                                          loss_meter=loss_meter,
                                                          accuracy=accuracy)))

    if (epoch) % val_every == 0:     
        print('Start validation #{}'.format(epoch+1))
        val_loss_meter, val_mIoU, val_mAcc, val_allAcc= validate(val_loader, model, criterion)
        if val_mIoU > best_mIoU:
            print('Best performance at epoch: {}'.format(epoch + 1))
            print('Save model in', saved_dir)
            best_mIoU = val_mIoU
            save_model(model, saved_dir)
        
#         wandb.log({
#             'train_loss': main_loss_meter.val,
#             'train_mIoU': aux_loss_meter,
#             'train_mAcc': loss_meter,
#             'train_allAcc': accuracy,
#         })
                
    iou_class = intersection_meter.sum / (union_meter.sum + 1e-10)
    accuracy_class = intersection_meter.sum / (target_meter.sum + 1e-10)
    mIoU = np.mean(iou_class)
    mAcc = np.mean(accuracy_class)
    allAcc = sum(intersection_meter.sum) / (sum(target_meter.sum) + 1e-10)
    
    return main_loss_meter.avg, mIoU, mAcc, allAcc

In [18]:
def validate(val_loader, model, criterion):
    loss_meter = AverageMeter()
    intersection_meter = AverageMeter()
    union_meter = AverageMeter()
    target_meter = AverageMeter()
    
    classes = 12
    print_freq = 50
    index_split = 0

    model.eval()
    for i, (input, target, _) in enumerate(val_loader):
        
        input = torch.stack(input)       # (batch, channel, height, width)
        target = torch.stack(target).long()
        
        output = model(input)
        if config.zoom_factor != 8:
            output = F.interpolate(output, size=target.size()[1:], mode='bilinear', align_corners=True)
        loss = criterion(output, target)

        n = input.size(0)
        loss = torch.mean(loss)

        output = output.max(1)[1]
        intersection, union, target = intersectionAndUnionGPU(output, target, classes, config.ignore_label)
        
        intersection, union, target = intersection.cpu().numpy(), union.cpu().numpy(), target.cpu().numpy()
        intersection_meter.update(intersection), union_meter.update(union), target_meter.update(target)

        accuracy = sum(intersection_meter.val) / (sum(target_meter.val) + 1e-10)
        loss_meter.update(loss.item(), input.size(0))

    iou_class = intersection_meter.sum / (union_meter.sum + 1e-10)
    accuracy_class = intersection_meter.sum / (target_meter.sum + 1e-10)
    mIoU = np.mean(iou_class)
    mAcc = np.mean(accuracy_class)
    allAcc = sum(intersection_meter.sum) / (sum(target_meter.sum) + 1e-10)
    
    print('Loss {loss_meter.val:.4f} ({loss_meter.avg:.4f}) '
          'mIoU {mIoU:.4f} '
          'Accuracy {accuracy:.4f}.'.format(loss_meter=loss_meter,
                                            mIoU = mIoU,
                                            accuracy=allAcc))
    
#     wandb.log({
#         'val_loss': main_loss_meter.val,
#         'val_mIoU': aux_loss_meter,
#         'val_mAcc': loss_meter,
#         'val_allAcc': allAcc,
#     })
        
    return loss_meter.avg, mIoU, mAcc, allAcc

## 모델 저장 함수 정의

In [19]:
saved_dir = '/opt/ml/code/saved'
file_name = 'pspnet_best_model(pretrained).pt'

if not os.path.isdir(saved_dir):                                                         
    os.mkdir(saved_dir)
    
def save_model(model, saved_dir, file_name=file_name):
    check_point = {'net': model.state_dict()}
    output_path = os.path.join(saved_dir, file_name)
    torch.save(model.state_dict(), output_path)

## 모델 생성 및 Loss function, Optimizer 정의

In [20]:
# Loss function 정의
criterion = nn.CrossEntropyLoss(ignore_index=225)

# 모델 생성
classes = 12
model = PSPNet(layers=50, classes=classes, zoom_factor=config.zoom_factor, criterion=criterion)

# Optimizer 정의
optimizer = torch.optim.SGD(model.parameters(), lr=config.base_lr, momentum=config.momentum, weight_decay=config.weight_decay)

config.criterion = "CrossEntropyLoss(ignore_index=225)"
config.optimizer = "SGD"

In [ ]:
# wandb.alert(title="Done", text="Train done.")
run.finish()

## 저장된 model 불러오기 (학습된 이후) 

In [ ]:
# best model 저장된 경로
model_path = os.path.join('/opt/ml/code/saved', file_name)

# best model 불러오기
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint)

# 추론을 실행하기 전에는 반드시 설정 (batch normalization, dropout 를 평가 모드로 설정)
model.eval()

save_model = model
model = model.cuda()

## submission을 위한 test 함수 정의

In [ ]:
def test(model, data_loader, device):
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    model.eval()
    
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    
    with torch.no_grad():
        for step, (imgs, image_infos) in enumerate(data_loader):

            # inference (512 x 512)
            outs = model(torch.stack(imgs).to(device))
            
            if outs.shape[0] > 1:
                oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
            else:
                oms = torch.argmax(outs, dim=1).detach().cpu().numpy()
            
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)

            oms = np.array(temp_mask)
            
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            preds_array = np.vstack((preds_array, oms))
            
            file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    return file_names, preds_array

## submission.csv 생성

---

In [ ]:
len(test_loader)

In [ ]:
lst_test_loader = list(test_loader)

In [ ]:
i = 0
print('type', type(lst_test_loader[i]), '| len', len(lst_test_loader[i]))
print()

print('type', type(lst_test_loader[i][0]), '| len', len(lst_test_loader[i][0]))

for j in range(len(lst_test_loader[i][0])):
    print(j,'- type', type(lst_test_loader[i][0][j]), '| shape', lst_test_loader[i][0][j].shape)
print()

print(lst_test_loader[i][1])

In [ ]:
i = 209
print('type', type(lst_test_loader[i]), '| len', len(lst_test_loader[i]))
print()

print('type', type(lst_test_loader[i][0]), '| len', len(lst_test_loader[i][0]))

for j in range(len(lst_test_loader[i][0])):
    print(j,'- type', type(lst_test_loader[i][0][j]), '| shape', lst_test_loader[i][0][j].shape)
print()

print(lst_test_loader[i][1])

In [ ]:
# 첫번째 batch의 추론 결과 확인
for imgs, image_infos in test_loader:
    image_infos = image_infos
    temp_images = imgs
    
    model.eval()
    # inference
    outs = model(torch.stack(temp_images).to(device))
    oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
    
    break

In [ ]:
i = 0
imgs = lst_test_loader[i][0]

outs = model(torch.stack(imgs).to(device))
print('outs', outs.shape)

oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
print('oms', oms.shape)

for img, mask in zip(np.stack(temp_images), oms):
    print(img.shape, mask.shape)

In [ ]:
i = 0
imgs = lst_test_loader[i][0]

outs = model(torch.stack(imgs).to(device))
print('outs', outs.shape)

oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
print('oms', oms.shape)

for img, mask in zip(np.stack(temp_images), oms):
    print(img.shape, mask.shape)

In [ ]:
i = 209
imgs = lst_test_loader[i][0]

outs = model(torch.stack(imgs).to(device))
print('outs', outs.shape[0])

oms = torch.argmax(outs.squeeze(), dim=0).detach().cpu().numpy()
print('oms', oms.shape)

for img, mask in zip(np.stack(temp_images), oms):
    print(img.shape, mask.shape)

In [ ]:
i = 209
imgs = lst_test_loader[i][0]

outs = model(torch.stack(imgs).to(device))
print('outs', outs.shape)

oms = torch.argmax(outs, dim=1).detach().cpu().numpy()
print('oms', oms.shape)

for img, mask in zip(np.stack(temp_images), oms):
    print(img.shape, mask.shape)

---

In [ ]:
# sample_submisson.csv 열기
submission = pd.read_csv('/opt/ml/code/submission/sample_submission.csv', index_col=None)

# test set에 대한 prediction
file_names, preds = test(model, test_loader, device)

# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
file_name = 'PSPNet(pretrained).csv'
submission.to_csv(os.path.join("/opt/ml/code/submission/", file_name), index=False)

## Check model

In [ ]:
# resnet = models.resnet50(pretrained=True)

In [ ]:
# nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool)

In [ ]:
# resnet.layer4

In [ ]:
# nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu, resnet.conv2, resnet.bn2, resnet.relu, resnet.conv3, resnet.bn3, resnet.relu, resnet.maxpool)